In [6]:
#_*_coding: utf-8_*_
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION
import sqlalchemy
import psycopg2
import time
import os

In [7]:
conn_string = os.environ['PG_DB']
cur_path = os.getcwd()
ser = Service(cur_path+'/chromedriver')

In [8]:
df_k = pd.read_excel("keywords.xlsx", sheet_name=None)
list_word=[]
for project in df_k.keys():
    display(df_k[project]['Статистика по словам'][0])
    list_word.append(df_k[project]['Статистика по словам'][0])
#display(list_word)
#list_word.append(df_k[project]['Статистика по словам'][0])
display(list_word)

'оранжевая корова'

'зебра +в клеточку'

'монсики'

'тайны медовой долины'

'чучи мяучи'

...

'чуддики'

'шиммер +и шайн'

'щенячий патруль'

'юху +и +его друзья'

['оранжевая корова',
 'зебра +в клеточку',
 'монсики',
 'тайны медовой долины',
 'чучи мяучи',
 'енотки',
 'ну погоди каникулы',
 'крутиксы',
 'простоквашино',
 'мартышкины мультфильм',
 'приключения пети +и волка',
 'умка мультфильм',
 'барбоскины',
 'добр бобр',
 'бодо бородо',
 'буба',
 'губка боб',
 'деревяшки',
 'дракоша тоша',
 'йоко мультик',
 'кошечки собачки',
 'лекс +и плу',
 'лео +и тиг',
 'лунтик',
 'маша +и медведь',
 'мимимишки',
 'сказочный патруль',
 'спина +к спине мультик',
 'три кота',
 'фееринки',
 'цветняшки',
 'мультсериал черепашки',
 'смешарики',
 'фиксики',
 'малышарики',
 '44 котенка',
 'мультик бен тен',
 'cry babies magic tears',
 'enchantimals мультик',
 'мультсериал take +my muffin',
 'приключения ам няма',
 'мультик белка +и стрелка',
 'буренка даша',
 'время приключений мультсериал',
 'герои энвелла',
 'геройчики',
 'мультики говорящий кот +том',
 'мультик гормити',
 'гравитю фолз',
 'джинглики',
 'диносити',
 'доктор плюшева',
 'звездная принцесса +и си

In [14]:
options = webdriver.ChromeOptions()
# get source code
df_list=[]
for keyword in list_word:
    options.add_argument("--user-data-dir="+cur_path+"/profile")
    browser=webdriver.Chrome(service=ser, options=options)
    #https://wordstat.yandex.ru/#!/history?words=простоквашино
    browser.get(r'https://wordstat.yandex.ru/#!/history?words='+keyword)
    time.sleep(15)
    html = browser.page_source
    time.sleep(1)
    browser.close()
    dfs = pd.read_html(html)
    df_1 = dfs[8]
    df_1 = df_1.drop(df_1.columns[[1,3]], axis = 1)
    df_1 = df_1.drop(0, axis = 0)
    df_2 = dfs[10]
    df_2 = df_2.drop(df_2.columns[[1,3]], axis = 1)
    df_2 = df_2.drop(0, axis = 0)
    df = pd.concat([df_1, df_2])
    df = df.reset_index()
    df["request"] = keyword
    df = df.drop(df.columns[[0]], axis = 1)
    df_list.append(df)
df_all=pd.concat(df_list)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
0   chromedriver                        0x0000000106efd138 chromedriver + 4923704
1   chromedriver                        0x0000000106e759d3 chromedriver + 4368851
2   chromedriver                        0x0000000106abe787 chromedriver + 472967
3   chromedriver                        0x0000000106a94c15 chromedriver + 302101
4   chromedriver                        0x0000000106b314cf chromedriver + 943311
5   chromedriver                        0x0000000106b462c6 chromedriver + 1028806
6   chromedriver                        0x0000000106b2bd23 chromedriver + 920867
7   chromedriver                        0x0000000106af2a4b chromedriver + 686667
8   chromedriver                        0x0000000106af4044 chromedriver + 692292
9   chromedriver                        0x0000000106ec88fe chromedriver + 4708606
10  chromedriver  

In [12]:
display(df_all)

NameError: name 'df_all' is not defined

In [5]:
df_all['date_from']= df_all['Период'].str.split('-', n=1, expand=True)[0].str.strip()
df_all['date_to']= df_all['Период'].str.split('-', n=1, expand=True)[1].str.strip()
df_all.date_from = pd.to_datetime(df_all.date_from,  format = '%d.%m.%Y')
df_all.date_to = pd.to_datetime(df_all.date_to, format = '%d.%m.%Y')

In [6]:
df_all = df_all.reset_index()

In [10]:
display(df_all)

,index,Период,Абсолютное,request,date_from,date_to
0,0,01.02.2021 - 28.02.2021,150286.0,оранжевая корова,2021-02-01,2021-02-28
1,1,01.03.2021 - 31.03.2021,158857.0,оранжевая корова,2021-03-01,2021-03-31
2,2,01.04.2021 - 30.04.2021,161684.0,оранжевая корова,2021-04-01,2021-04-30
3,3,01.05.2021 - 31.05.2021,155542.0,оранжевая корова,2021-05-01,2021-05-31
4,4,01.06.2021 - 30.06.2021,150960.0,оранжевая корова,2021-06-01,2021-06-30
...,...,...,...,...,...,...
1843,19,01.09.2022 - 30.09.2022,6756.0,юху +и +его друзья,2022-09-01,2022-09-30
1844,20,01.10.2022 - 31.10.2022,8475.0,юху +и +его друзья,2022-10-01,2022-10-31
1845,21,01.11.2022 - 30.11.2022,7534.0,юху +и +его друзья,2022-11-01,2022-11-30
1846,22,01.12.2022 - 31.12.2022,8003.0,юху +и +его друзья,2022-12-01,2022-12-31


In [13]:
db = create_engine(conn_string)
conn = db.connect()

In [14]:
print("Reading yt_reels_stat table with latest date...")
df_old =  pd.read_sql("SELECT * FROM wordstat_monthly", conn, parse_dates=['date_from', 'date_to'])
print("Done.")

Reading yt_reels_stat table with latest date...
Done.


In [58]:
df_res = pd.concat([df_old, df_all])

In [65]:
df_res = df_res.drop_duplicates(subset = ['Период', 'request'], keep = 'last') 

In [68]:
#display(df_res)

In [11]:
inject_dict = { 'Период': sqlalchemy.types.TEXT(),  
                'Абсолютное': sqlalchemy.types.BIGINT(), 
                'request': sqlalchemy.types.TEXT(),  
                'date_from': sqlalchemy.types.Date(),                   
                'date_to': sqlalchemy.types.Date(), 
                 }
start_time = time.time()
res = df_all.to_sql('wordstat_monthly', con=conn, if_exists='replace', index=False, dtype = inject_dict)
print("to_sql duration: {} seconds".format(time.time() - start_time))

to_sql duration: 1.855560064315796 seconds
